In [48]:
import matplotlib.pyplot as plt
import matplotlib
import sqlite3
import json
import numpy as np
from imgurpython import ImgurClient
from matplotlib.font_manager import FontProperties 

font = FontProperties(fname=r"./simhei.ttf", size=18)

def heatmap(data, row_labels, col_labels, **kwargs):
    ax = plt.gca()
    im = ax.imshow(data, **kwargs)
    
    ax.set_xticks(np.arange(data.shape[1]))
    ax.set_yticks(np.arange(data.shape[0]))
    ax.set_xticklabels(col_labels, fontproperties=font)
    ax.set_yticklabels(row_labels, size=18)
    ax.tick_params(top=True, bottom=False, labeltop=True, labelbottom=False)
    plt.setp(ax.get_xticklabels(), rotation=0, ha="center", rotation_mode="anchor")

    for edge, spine in ax.spines.items():
        spine.set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=2)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im

def annotate_heatmap(im, valfmt, textcolors=("black", "white"), **textkw):
    data = im.get_array()
    threshold = im.norm(data.max())/2.
    kw = dict(horizontalalignment="center", verticalalignment="center")
    kw.update(textkw)
    valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[int(im.norm(data[i,j]) > threshold)])
            text = im.axes.text(j, i, valfmt(data[i,j], None), **kw, size=20)
            texts.append(text)

    return texts

In [52]:
def gen_plot(cursor):
    cursor.execute('select user, source from raw_data where source != \'\'')
    rawdata = cursor.fetchall()

    users, sourcelist = {}, {}
    for data in rawdata:
        if data[0] in users:
            if data[1] in users[data[0]]:
                users[data[0]][data[1]] += 1
            else:
                users[data[0]][data[1]] = 1
                sourcelist.update(source)
        else:
            source = {}
            source[data[1]] = 1
            sourcelist.update(source)
            users[data[0]] = source

    n = np.zeros([len(sourcelist), len(users)])
    for i, source in enumerate(sourcelist):
        for j, user in enumerate(users):
            try:
                n[i,j] = users[user][source]
            except:
                n[i,j] = 0
            
    plt.figure(figsize=(9,16))
    im = heatmap(n, sourcelist, users, cmap="Blues")
    texts = annotate_heatmap(im, valfmt="{x:.0f}")
    plt.savefig('./statisticsimage.png')
    imagelink, deletehash = upload()
    
    return imagelink, deletehash

def upload():
    with open('Iconfig.json', 'r')as f:
        j = json.load(f)

    client = ImgurClient(j['client_id'], j['client_secret'], j['access_token'], j['refresh_token'])
    image = client.upload_from_path('statisticsimage.png', config=None, anon=False)
    
    return image['link'], image['deletehash']
